# Create custom email notifications for new listings

**Based on user critieras, an email will be sent to your email once a week with the newest listings.**

In [120]:
# Import necessary libraries
import requests
import time
import random
import string
import hashlib
import pandas as pd
import json
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders
import csv

**This script will enable you to create custom email alerts for new house listings.** 
<br> You need your own CallerId for the Booli API. </br>

In [121]:
user_email = input("Enter your email address: ")

with open('users.csv', 'r') as f:
    reader = csv.reader(f)
    user_list = list(reader)
    
user_list_2 = [item for user_list_2 in user_list for item in user_list_2]
    
if user_email in user_list_2:
    print("You already have an alarm config.")
    answer = input("Do you want to reset? y/n: ")
    if answer == 'n':
        print("EXIT")
        exit()
        
    else:
        print("Continue")
        pass
    

print("append to list")
user_list_2.append(user_email)
    
with open('users.csv', 'w') as f:
    wr = csv.writer(f, quoting=csv.QUOTE_ALL)
    wr.writerow(user_list_2)


Enter your email address: test
You already have an alarm config.
Do you want to reset? y/n: y
Continue
append to list


In [122]:
# Ask user for API login information

callerId = input("Please enter your Booli-API CallerId: ")
if len(callerId) == 0:
    callerId = input("You did not write anything! Please enter your Booli-API CallerId: ")

user_key = 'ENTER-API-PASSWORD'
#key = input("Please enter your Booli-API private key: ")
#if len(key) == 0:
    #key = input("You did not write anything! Please enter your Booli-API private key: ")

print("Perfect! Now let´s enter some custom configs for your email notifications.")

Please enter your Booli-API CallerId: Rikard Fahlström
Perfect! Now let´s enter some custom configs for your email notifications.


In [123]:
# Ask for custom search options

# Define area
area = input("Which area are you interested in? Type for e.g. Uppsala, Falun, Vasastan. ")
if len(area) == 0:
    area = input("Please enter area of interest: ")

# Define type_of_object    
valid_type_of_objects = ['villa', 'lägenhet', 'gård', 'tomt-mark', 'fritidshus', 'parhus','radhus','kedjehus']    

type_of_object = input("What type of object are you interested in? Enter 'villa', 'lägenhet', 'gård', 'tomt-mark', 'fritidshus', 'parhus','radhus' or 'kedjehus'. ")
if type_of_object in valid_type_of_objects:
    print("Thanks!")
else:
    type_of_object = input("Please enter which type of object you are interested in. Enter one of following; 'villa', 'lägenhet', 'gård', 'tomt-mark', 'fritidshus', 'parhus','radhus' or 'kedjehus'.")

# Define minimum number of rooms
MinRooms = input("Enter minimum number of rooms: ")
if len(MinRooms) == 0:
    MinRooms = input("Please enter minimum number of rooms as single digit.")
elif MinRooms == "0":
    MinRooms = input("0 number of rooms is not a valid object, please enter number of rooms >0: ")
    

# Ask for maximum list price
MaxPrice = input("Enter maximum list price for objects: ")
if len(MaxPrice) == 0:
    MaxPrice = input("Please enter the maximum list price: ")


print("Great, you will now receive a email once per day.")

Which area are you interested in? Type for e.g. Uppsala, Falun, Vasastan. Falun
What type of object are you interested in? Enter 'villa', 'lägenhet', 'gård', 'tomt-mark', 'fritidshus', 'parhus','radhus' or 'kedjehus'. lägenhet
Thanks!
Enter minimum number of rooms: 2
Enter maximum list price for objects: 5000000
Great, you will now receive a email once per day.


In [124]:
# Create API-requests based on user input

# Inputvärden
callerId = "Rikard Fahlström"
time = str(time.time())
unique = str(''.join(random.choice(string.ascii_lowercase + string.digits) for _ in range(16)))
key = user_key
hashstr = hashlib.sha1((callerId+time+key+unique).encode('utf-8')).hexdigest()
query_type = 'listings'  # Enter 'listings', 'sold' eller 'areas'
query_city = area
offset = '0' # 
limit = str(10)  # Max antal svar från deras server. Maxgräns = 500
object_type = str(type_of_object)

# Själva API-frågan
r = requests.get("https://api.booli.se/"+
                 query_type+"?q="+query_city+
                 "&callerId="+callerId+
                 "&time="+time+
                 "&unique="+unique+
                 "&hash="+str(hashstr)+
                 "&limit="+limit+
                 "&offset="+offset+
                 "&objectType="+object_type+
                "&minRooms="+MinRooms+
                "&maxListPrice="+MaxPrice)

print("Status code from API request = " + str(r.status_code)+ "\n")

if r.status_code != 200:
    print('fail')
    exit()

output = json.loads(r.text)

print("Number of listings returned in API = " + str(len(output['listings'])) + "\n")
print(output.keys())

Status code from API request = 200

Number of listings returned in API = 10

dict_keys(['totalCount', 'count', 'listings', 'limit', 'offset', 'searchParams'])


In [125]:
urls = []
for value in range(len(output['listings'])):
    urls.append(output['listings'][value]['url'])

#print(type(urls))

In [126]:
object_id = []
for value in range(len(output['listings'])):
    object_id.append(output['listings'][value]['booliId'])

#print(type(object_id))

In [127]:
room_info = []
for value in range(len(output['listings'])):
    room_info.append(output['listings'][value]['rooms'])

#print(room_info)

In [128]:
muni_info = []
for value in range(len(output['listings'])):
    muni_info.append(output['listings'][value]['location']['region']['municipalityName'])

#print(muni_info)

In [129]:
price_info = []
for value in range(len(output['listings'])):
    price_format = '{:,}'.format(output['listings'][value]['listPrice'])
    price_info.append(price_format)

#print(price_info)

In [130]:
address_info = []
for value in range(len(output['listings'])):
    address_info.append(output['listings'][value]['location']['address']['streetAddress'])

#print(address_info)

In [131]:
published_info = []
for value in range(len(output['listings'])):
    date_format = pd.to_datetime(output['listings'][value]['published'],format="%Y-%m-%d %H:%M:%S")
    published_info.append(date_format.date())
    
#print(published_info)

In [132]:
df = pd.DataFrame({
    'Published':published_info,
     '2. Area':muni_info,
     '1. Address':address_info,
     '4. Rooms':room_info,
     '3. Price':price_info,
    '5. urls':urls
     })

df2 = df.set_index(['Published'])
df_html = df2.to_html()

df2

,1. Address,2. Area,3. Price,4. Rooms,5. urls
Published,,,,,
2017-06-18,Herrhagsvägen 13C,Falun,"1,195,000",3.0,https://www.booli.se/annons/2355236
2017-06-16,Kungsgårdsvägen 2A,Falun,"1,150,000",3.0,https://www.booli.se/annons/2354781
2017-06-16,Linnévägen 23A,Falun,"1,825,000",2.5,https://www.booli.se/annons/2354008
2017-06-14,Linnévägen 23A,Falun,"1,595,000",2.0,https://www.booli.se/annons/2353076
2017-06-13,Ingels väg 147,Falun,"1,350,000",2.0,https://www.booli.se/annons/2352350
2017-06-11,Sturegatan 75E,Falun,"1,495,000",3.0,https://www.booli.se/annons/2346310
2017-06-10,Drottningvägen 10B,Falun,"350,000",3.0,https://www.booli.se/annons/2350503
2017-06-03,Linnévägen 29A,Falun,"1,925,000",3.0,https://www.booli.se/annons/2345731
2017-06-02,Hyttgatan 33A,Falun,"1,385,000",3.0,https://www.booli.se/annons/2345463


In [133]:
html = """\
       <html>
            <body>
                
                Hi,
                <p>
                    Here is your custom email notification regarding new listings. 
                    <b></b>
                    %s
                    <b></b>
                    Please contact rikard.fahlstrom@etraveli.com if you have suggestions for improvements.
                </p> 
             </body>
       </html>
   """ %(df_html)

In [135]:
# Add df to email

fromaddr = 'Custom email notification'
toaddr = 'rikard.fahlstrom@etraveli.com'
subject = 'Most recent listings'
msg = MIMEMultipart()

msg['From'] = fromaddr
msg['To'] = toaddr
msg['Subject'] = subject
msg.attach(MIMEText(html, 'html'))


server = smtplib.SMTP('smtp.gmail.com', 587)
server.starttls()
server.login('rikard.email.notification@gmail.com', 'ENTER-EMAIL-PASSWORD')
text = msg.as_string()
server.sendmail('rikard.email.notification@gmail.com', toaddr, text)
server.quit()

print('Done')

Done
